<a href="https://colab.research.google.com/github/Anjana71/image_classifier/blob/main/image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio torchvision torch --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.7 MB/s eta 0:00:00


In [2]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import urllib.request

# Load model
model = models.resnet18(pretrained=True)
model.eval()

# Download ImageNet class labels
LABELS_URL = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
urllib.request.urlretrieve(LABELS_URL, "imagenet_classes.txt")
labels = [line.strip() for line in open("imagenet_classes.txt")]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 309MB/s]


In [3]:
import random

# Optional: Some emojis for flair
emoji_map = {
    "dog": "🐶", "cat": "🐱", "car": "🚗", "bird": "🐦", "flower": "🌸", "tree": "🌳", "computer": "💻",
    "phone": "📱", "pizza": "🍕", "person": "🧑", "banana": "🍌", "bottle": "🍾", "cup": "☕"
}

def get_emoji(label):
    for key in emoji_map:
        if key in label.lower():
            return emoji_map[key]
    return "✨"  # default emoji


In [4]:
def classify_image(img):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    img_tensor = transform(img).unsqueeze(0)

    with torch.no_grad():
        output = model(img_tensor)
        probs = torch.nn.functional.softmax(output[0], dim=0)

    top5_prob, top5_catid = torch.topk(probs, 5)

    results = []
    for i in range(top5_prob.size(0)):
        label = labels[top5_catid[i]]
        confidence = top5_prob[i].item()
        emoji = get_emoji(label)
        results.append(f"{emoji} **{label}** — {confidence:.2%}")

    return "\n\n".join(results)


In [ ]:
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("## 🧠 AI Image Classifier — *Top 5 Predictions*")
    gr.Markdown("Upload any image and get the top-5 predicted classes with confidence and emojis 🎉")

    with gr.Row():
        image_input = gr.Image(type="pil", label="Upload an Image")
        output_text = gr.Markdown(label="AI Predictions")

    image_input.change(fn=classify_image, inputs=image_input, outputs=output_text)

    gr.Markdown("---")
    gr.Markdown("Created with ❤️ using ResNet18 & HuggingFace")

app.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0223adfdae83b4dc47.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
